In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from SyntheticControlMethods import Synth
import synth_utils as su
import pandas as pd
import numpy as np
import synth_job as sj
import Dino.cf_value as cfv
import Dino.counterfeit_utils as cfu

In [12]:
slugs = cfu.get_top_slugs(cfu.CUT_OFF,cfu.DB_NAME)

#Get all looksims
for slug in slugs:
    try:
        df = su.create_average_synth_job_df(slug)
    except:
        continue
    #pickle df into job_files directory
    alts  = list(df['Alt'])
    if len(alts) < 5:
        continue
    df.to_pickle(f'job_files/{slug}_avg_synth_df.pkl')
    sj.make_and_save_alt_day_dict(alts,filename=f'job_files/{slug}_alt_day_dict.pkl')
    sj.write_slurm_script(slug, len(alts))


No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
Slurm script generated and saved to: slurm_scripts/slurm_veefriends-series-2.sh
No volume data for slug
Slurm script generated and saved to: slurm_scripts/slurm_wonderpals.sh
Slurm script generated and saved to: slurm_scripts/slurm_world-of-women-galaxy.sh
Slurm script generated and saved to: slurm_scripts/slurm_world-of-women-nft.sh


In [10]:
slugs[25:]

['kprverse',
 'lilpudgys',
 'livesofasuna',
 'metahero-generative',
 'mfers',
 'milady',
 'moonbirds-oddities',
 'muribyfabrik',
 'mutant-ape-yacht-club',
 'nakamigos',
 'official-v1-punks',
 'phantabear',
 'proof-moonbirds',
 'pudgypenguins',
 'pxnghostdivision',
 'rare-apepes',
 'remilio-babies',
 'sappy-seals',
 'sneaky-vampire-syndicate',
 'supducks',
 'the-crypto-chicks',
 'thepotatoz',
 'theshiboshis',
 'tubby-cats',
 'veefriends',
 'veefriends-series-2',
 'wonderpals',
 'world-of-women-galaxy',
 'world-of-women-nft']

In [8]:
slugs = cfu.get_top_slugs(cfu.CUT_OFF,cfu.DB_NAME)
for slug in slugs:
    try:
        job_file = f'{slug}_avg_synth_df.pkl'
        results = sj.get_all_placebo_results(job_file,slug)
        print(slug,hoeffding_confidence_interval(results['p_value'], delta=0.05))
        print(empirical_bernstein_confidence_interval(results['p_value'], delta=0.05))
        print(len(results['p_value']))
    except:
        continue

0n1-force (0.273847593521185, 1)
(0.2371921800110599, 1.1294690351994163)
11
3landers (0.11845017592000234, 0.7772261758858869)
(0.11869670294143486, 0.7769796488644546)
17
alienfrensnft (0.18683223937338117, 1)
(0.1352841473837011, 1.0573463595311674)
11
azuki (0.07498399256543237, 0.6413514761347932)
(0.05424916899722515, 0.6620862997030001)
23
beanzofficial (0, 0.7481237861887908)
(-0.3248031184351907, 0.8607278994928516)
8
boredapeyachtclub (0.15514886431077868, 0.7095914863882677)
(0.1561711312368504, 0.7085692194621958)
24
chain-runners-nft (0, 1)
(-0.20249087027044882, 1.1775924258710437)
7
clonex (0, 0.9210820486092903)
(-0.2283464260995779, 1.0438822759382864)
7
cool-cats-nft (0.2862075823961251, 0.8789316773785679)
(0.26830851813410683, 0.8968307416405862)
21
coolmans-universe (0.09252815552988986, 0.9514669722233651)
(0.0405250871259315, 1.0034700406273234)
10
cryptodickbutts-s3 (0, 0.7376094396360535)
(-0.11066079543016288, 0.8017793990025845)
12
cryptopunks (0.157083221857

In [7]:
def empirical_bernstein_confidence_interval(X, delta=0.05, b=1.0, tol=1e-12, max_iter=200):
    """
    Returns a (lower, upper) confidence interval for the mean of X
    using an empirical Bernstein bound.
    
    The bound ensures:
        P( |mean(X) - true_mean| >= eps ) <= delta
    
    :param X:       list/array of samples (each in [0, b] if b=1)
    :param delta:   desired error probability
    :param b:       bound on max |X_i - mean| (often 1 if X_i in [0,1])
    :param tol:     tolerance for numerical search
    :param max_iter:max iterations for binary search
    
    :return: (L, U) = (mean(X) - eps, mean(X) + eps)
    """
    X = list(X)
    n = len(X)
    if n < 2:
        raise ValueError("Need at least 2 samples to estimate variance.")
    
    # Empirical mean
    mean_emp = sum(X) / n
    
    # Empirical variance (unbiased sample variance)
    # can also use math.variance(X) in Python >=3.8
    var_emp = sum((x - mean_emp)**2 for x in X) / (n - 1)
    
    # We'll solve for eps in:
    #
    # 2 * exp( - n * eps^2 / (2 * var_emp + (2/3)*b * eps) ) = delta
    #
    # We'll define a function LHS(eps) and do LHS(eps) - delta <= 0.
    def bernstein_lhs(eps):
        denom = 2.0 * var_emp + (2.0/3.0) * b * eps
        return 2.0 * math.exp(- (n * eps*eps) / denom)
    
    # We do a simple binary search for eps >= 0
    # bracket from [0, big] until LHS(big) < delta
    Emin, Emax = 0.0, 1.0
    while bernstein_lhs(Emax) > delta:
        Emax *= 2.0
        if Emax > 1e10:
            raise ValueError("Could not bracket the solution. Possibly data or delta is degenerate.")
    
    for _ in range(max_iter):
        Emid = 0.5 * (Emin + Emax)
        val_mid = bernstein_lhs(Emid)
        if val_mid > delta:
            # need to increase eps to drive LHS down
            Emin = Emid
        else:
            Emax = Emid
        if (Emax - Emin) < tol:
            break
    
    eps = 0.5 * (Emin + Emax)
    return (mean_emp - eps, mean_emp + eps)


In [4]:
import math
def hoeffding_confidence_interval(data, delta=0.05):
    """
    Returns a (lower, upper) confidence interval for the mean of 'data'
    using Hoeffding's inequality at confidence level 1 - delta.
    """
    data = np.array(data)
    n = len(data)
    sample_mean = np.mean(data)
    
    # Hoeffding bound for [0,1] random variables:
    #   P(|sample_mean - true_mean| >= epsilon) <= 2*exp(-2*n*epsilon^2)
    # Solve for epsilon when the RHS = delta:
    #   2*exp(-2*n*epsilon^2) = delta  ==>  epsilon = sqrt(ln(2/delta)/(2*n))
    epsilon = math.sqrt( math.log(2.0/delta) / (2.0 * n) )
    
    lower = sample_mean - epsilon
    upper = sample_mean + epsilon
    
    # You might optionally clip the interval to [0,1] if you know the true mean must be in [0,1].
    lower = max(0, lower)
    upper = min(1, upper)

    return lower, upper

In [7]:
df = pd.read_pickle('sales.pkl')
df['date'] = cfu.timestamps_to_dates(df['timestamp'],raw_timestamps=True)

In [5]:
slug = 'azuki'
df = pd.read_pickle(f'job_files/{slug}_alt_day_dict.pkl')
df

{'0n1-force': 1514,
 'valhalla': 1887,
 'muribyfabrik': 1724,
 '77bit': 1768,
 'kprverse': 1958,
 'divineanarchy': 1596,
 'avariksagauniverse': 1553,
 'dezuki': 1706,
 'kokodinft': 1683,
 'official-aneroverse': 1818,
 'ether': 2210,
 'edo2022': 1839,
 'projectatamaofficial': 1871,
 'sofa-vision': 2341,
 'neo-samurai-monkeys': 1790,
 'akyllersden': 1904,
 'eien-origin': 1931,
 'hyakkiofficial': 1975,
 'projectatamasoldiers': 2052,
 'inakatabi': 2002,
 'denizens-of-zenacademy': 2257,
 'convictionstd': 2322,
 'swordsmannft': 2075,
 'keeei-goat-buddys': 2027,
 'mutant-azuki': 1740,
 'dickzuki': 2296,
 'senshi-6': 2112}

In [6]:
slugs = cfu.get_top_slugs(cfu.CUT_OFF,cfu.DB_NAME)
master_df = pd.DataFrame()
#Get all looksims
for slug in slugs:
    try:
        df = su.create_average_synth_job_df(slug,overlap=1,return_dates=True)
    except:
        continue
    master_df = pd.concat([master_df,df])
display(master_df)
    

No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data for slug
No volume data f

,Top_100,Alt,alt_date
0,0n1-force,0xvampire-project,2021-09-15
1,0n1-force,0xzuki,2022-02-04
2,0n1-force,tokyo-brave-heroes,2022-06-29
3,0n1-force,roars-by-ntp,2023-04-28
4,0n1-force,mekazuki,2022-03-12
...,...,...,...
4,world-of-women-nft,chillennials,2022-09-30
5,world-of-women-nft,asian-identity-collection,2022-04-22
6,world-of-women-nft,world-of-boys-official,2022-01-30
7,world-of-women-nft,stargals,2022-06-04


In [7]:
master_df.to_pickle('master_df.pkl')

In [ ]:
mast_df